In [20]:
import tensorflow as tf

In [21]:
import numpy as np
import pandas as pd

In [22]:
df=pd.read_csv('/home/saloni/Documents/ML/Mortality/train.csv')
labels=pd.read_csv('/home/saloni/Documents/ML/Mortality/labels_train.csv')


In [23]:
dailydata=df.copy()
result=pd.concat([dailydata,labels],axis=1)

In [24]:
result.head(1)

,ALP,ALT,AST,Age,Albumin,BUN,Bilirubin,Cholesterol,Creatinine,DiasABP,...,SaO2,SysABP,Temp,TroponinI,TroponinT,Urine,WBC,Weight,pH,In-hospital_death
0,77.0,31.0,46.0,54,2.973333,10.5,0.7,154.0,0.75,58.795833,...,97.25,116.891892,37.357143,2.1,0.14,171.052632,10.3,80.060976,7.387273,0


In [25]:
list_of_variables = ['Age', 'Albumin', 'ALP', 'ALT', 'AST', 'Bilirubin', 'BUN', 'Cholesterol', 'Creatinine',
                             'DiasABP', 'FiO2', 'GCS', 'Gender', 'Glucose', 'HCO3', 'HCT', 'Height', 'HR', 'ICUType', 'K',
                             'Lactate', 'Mg', 'MAP', 'MechVent', 'Na', 'NIDiasABP', 'NIMAP', 'NISysABP', 'PaCO2', 'PaO2',
                             'pH', 'Platelets', 'RecordID', 'RespRate', 'SaO2', 'SysABP', 'Temp', 'TroponinI','TroponinT','Urine',
                             'WBC', 'Weight']

In [26]:
datawithoutoutliers=result.copy()
    

In [27]:
for i in list_of_variables:
    datawithoutoutliers=datawithoutoutliers[np.abs(datawithoutoutliers[i]-datawithoutoutliers[i].mean())<=(4*datawithoutoutliers[i].std())]
    outliers=datawithoutoutliers[np.abs(datawithoutoutliers[i]-datawithoutoutliers[i].mean())>=(4*datawithoutoutliers[i].std())]
    
print(datawithoutoutliers.shape)

(3169, 43)


In [28]:
labels=datawithoutoutliers.pop("In-hospital_death")
df=datawithoutoutliers.copy()

In [29]:
labels.shape


(3169,)

In [30]:
df.shape

(3169, 42)

In [31]:
from sklearn.preprocessing import MinMaxScaler
#normalization
scaler = MinMaxScaler(feature_range=(0, 1))
df = scaler.fit_transform(df)


In [32]:
from imblearn.over_sampling import SMOTE

df_res,labels_res=SMOTE(random_state=9).fit_sample(df,labels) #balancing the data
df_res=pd.DataFrame(df_res)
labels_res=pd.DataFrame(labels_res)

In [33]:
from sklearn.preprocessing import OneHotEncoder
enc=OneHotEncoder(sparse=False)
onehot_encoded = enc.fit_transform(labels_res)

labels_res=onehot_encoded

In [34]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df_res,labels_res,test_size=0.33,random_state=42)


In [35]:
sess=tf.InteractiveSession()

In [36]:
n_hidden_1=120
n_hidden_2=120
n_hidden_3=120
n_hidden_4=120



x=tf.placeholder(tf.float32,shape=[None,42])
y_=tf.placeholder(tf.float32,shape=[None,2])
w =tf.Variable(tf.zeros([42,2]))
b=tf.Variable(tf.zeros([2]))

def multilayer_perceptron(x,weights,biases):
    layer_1=tf.add(tf.matmul(x,weights['h1']),biases['b1'])
    layer_1=tf.nn.sigmoid(layer_1)
    
    layer_2=tf.add(tf.matmul(layer_1,weights['h2']),biases['b2'])
    layer_2=tf.nn.sigmoid(layer_2)
    
    layer_3=tf.add(tf.matmul(layer_2,weights['h3']),biases['b3'])
    layer_3=tf.nn.sigmoid(layer_3)
    
    layer_4=tf.add(tf.matmul(layer_3,weights['h4']),biases['b4'])
    layer_4=tf.nn.sigmoid(layer_4)
    
    
    
    out_layer=tf.add(tf.matmul(layer_4,weights['out']),biases['out'])
    out_layer=tf.nn.relu(out_layer)
    return out_layer

weights={
       
    'h1':tf.Variable(tf.truncated_normal([42,n_hidden_1])),
    'h2':tf.Variable(tf.truncated_normal([n_hidden_1,n_hidden_2])),
    'h3':tf.Variable(tf.truncated_normal([n_hidden_2,n_hidden_3])),
    'h4':tf.Variable(tf.truncated_normal([n_hidden_3,n_hidden_4]),
    'out':tf.Variable(tf.truncated_normal([n_hidden_4,2]))
}

biases={
    'b1':tf.Variable(tf.truncated_normal([n_hidden_1])),
    'b2':tf.Variable(tf.truncated_normal([n_hidden_2])),
    'b3':tf.Variable(tf.truncated_normal([n_hidden_3])),
    'b4':tf.Variable(tf.truncated_normal([n_hidden_4])),
    'out':tf.Variable(tf.truncated_normal([2]))
}
    
y=multilayer_perceptron(x,weights,biases)
cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_,logits=y))  
train_step=tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
sess.run(tf.global_variables_initializer())                     
                     
    
    

SyntaxError: invalid syntax (<ipython-input-36-36c9713c9b49>, line 38)

In [ ]:
mse_history=[]
cost_history=[]
accuracy_history=[]

In [ ]:
training_epochs=1000

In [ ]:
for epoch in range(training_epochs):
    sess.run(train_step,feed_dict={x:x_train,y_:y_train})
    cost=sess.run(cross_entropy,feed_dict={x:x_train,y_:y_train})
    cost_history.append(np.append(cost_history,cost))
    correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
    accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    
    pred_y=sess.run(y,feed_dict={x:x_test})
    mse=tf.reduce_mean(tf.square(pred_y-y_test))
    mse_=sess.run(mse)
    mse_history.append(mse_)
    accuracy=sess.run(accuracy,feed_dict={x:x_train,y_:y_train})
    accuracy_history.append(accuracy)
    
    print('epoch:',epoch,'-','cost:',cost,'mse: ',mse_,'-','-train accuracy:',accuracy)

In [ ]:
correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
print('test accuracy:',sess.run(accuracy,feed_dict={x:x_test,y_:y_test}))

In [ ]:
pred_y=sess.run(y,feed_dict={x:x_test})
mse=tf.reduce_mean(tf.square(pred_y-y_test))
print('mse:%.4f'%sess.run(mse))